# 🧮 Magpie Math Dataset Generator - Google Colab Edition

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtani-y/magpie/blob/main/generate_colab.ipynb)

**HLE（高等レベル試験）数学対策に特化した最高難易度データセット生成システム**

## 🚀 特徴
- **🏆 最高難易度**: IMO、Putnam、PhD級の問題
- **🧠 超深層推論**: 10-20ステップの詳細証明（4096トークン）
- **6分野数学**: 代数・微積分・幾何・統計・数論・離散数学
- **6モデル対応**: DeepSeek R1、Qwen2.5、Gemma 3シリーズ
- **対話式操作**: 簡単な選択で44,000問の高品質データセット生成

## 📦 セットアップ

### ステップ1: 環境準備

In [ ]:
# 必要なライブラリのインストール
!pip install vllm==0.2.1.post1
!pip install transformers accelerate torch
!pip install ipywidgets

# Magpieリポジトリのクローン
!git clone https://github.com/Ohtani-y/magpie.git
%cd magpie

print("✅ セットアップ完了！")

### ステップ2: GPU確認

In [ ]:
import torch
import subprocess

# GPU情報の確認
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    print(f"🎯 GPU検出: {gpu_name}")
    print(f"📊 メモリ: {gpu_memory:.1f} GB")
    print(f"🔢 GPU数: {gpu_count}")
    
    # 推奨モデルの提案
    if gpu_memory >= 80:
        print("\n🏆 推奨モデル: DeepSeek R1 Distill Llama 70B または Qwen2.5-Math 72B")
    elif gpu_memory >= 32:
        print("\n⭐ 推奨モデル: DeepSeek R1 Distill Qwen 32B または Qwen2.5-Coder 32B")
    elif gpu_memory >= 24:
        print("\n💡 推奨モデル: Gemma 3 27B または DeepSeek R1 FP4")
    else:
        print("\n⚠️  警告: メモリ不足の可能性があります。FP4量子化モデルを使用してください。")
        
else:
    print("❌ GPU が検出されませんでした。ランタイムの種類をGPUに変更してください。")

## 🎯 対話式データセット生成

### ステップ3: モデル選択

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import os

# モデル定義
models = {
    "1": {
        "name": "DeepSeek R1 Distill Qwen 32B", 
        "path": "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
        "description": "バランス型（推奨） - V100 32GB+",
        "script": "magpie-deepseek-r1-distill-qwen-32b.sh"
    },
    "2": {
        "name": "DeepSeek R1 Distill Llama 70B", 
        "path": "deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
        "description": "高性能型 - A100 80GB",
        "script": "magpie-deepseek-r1-distill-llama-70b.sh"
    },
    "3": {
        "name": "DeepSeek R1 FP4", 
        "path": "deepseek-ai/DeepSeek-R1-0528-FP4",
        "description": "メモリ効率型 - RTX 4090+",
        "script": "magpie-deepseek-r1-fp4.sh"
    },
    "4": {
        "name": "Gemma 3 27B", 
        "path": "google/gemma-3-27b-it",
        "description": "Google最新モデル - V100 32GB+",
        "script": "magpie-gemma3-27b.sh"
    },
    "5": {
        "name": "Qwen2.5-Math 72B", 
        "path": "Qwen/Qwen2.5-Math-72B-Instruct",
        "description": "数学特化モデル - A100 80GB",
        "script": "magpie-qwen2.5-math-72b.sh"
    },
    "6": {
        "name": "Qwen2.5-Coder 32B", 
        "path": "Qwen/Qwen2.5-Coder-32B-Instruct",
        "description": "計算数学特化 - V100 32GB+",
        "script": "magpie-qwen25-coder-32b.sh"
    }
}

# モデル選択ウィジェット
model_options = [(f"{k}: {v['name']} - {v['description']}", k) for k, v in models.items()]
model_selector = widgets.Dropdown(
    options=model_options,
    value="1",
    description="モデル選択:",
    style={'description_width': 'initial'}
)

display(model_selector)

print("🎯 モデルを選択してください")

### ステップ4: 数学分野選択

In [ ]:
# 数学分野定義
domains = {
    "algebra": {
        "name": "代数学 (Algebra)",
        "recommended": 10000,
        "description": "抽象代数、ガロア理論、体拡大、行列理論"
    },
    "calculus": {
        "name": "微積分学 (Calculus)", 
        "recommended": 10000,
        "description": "実解析、複素解析、測度論、函数解析"
    },
    "geometry": {
        "name": "幾何学 (Geometry)",
        "recommended": 6000,
        "description": "射影幾何、微分幾何、代数幾何"
    },
    "statistics": {
        "name": "統計学 (Statistics)",
        "recommended": 6000,
        "description": "測度論的確率、マルチンゲール、高次統計理論"
    },
    "number_theory": {
        "name": "数論 (Number Theory)",
        "recommended": 4000,
        "description": "解析的数論、代数的数論、L関数"
    },
    "discrete": {
        "name": "離散数学 (Discrete Mathematics)",
        "recommended": 8000,
        "description": "極値グラフ理論、代数的組合せ論、計算複雑性"
    }
}

# 生成モード選択
mode_selector = widgets.RadioButtons(
    options=[
        ('単一分野生成', 'single'),
        ('全分野一括生成 (44K問題)', 'all'),
        ('カスタム選択', 'custom')
    ],
    value='single',
    description='生成モード:',
    style={'description_width': 'initial'}
)

display(mode_selector)

print("📚 生成モードを選択してください")

### ステップ5: 分野設定

In [ ]:
# 単一分野用ウィジェット
domain_options = [(f"{v['name']} ({v['recommended']:,}問推奨) - {v['description']}", k) 
                  for k, v in domains.items()]

single_domain_selector = widgets.Dropdown(
    options=domain_options,
    value="algebra",
    description="数学分野:",
    style={'description_width': 'initial'}
)

problem_count_input = widgets.IntText(
    value=10000,
    description="問題数:",
    style={'description_width': 'initial'}
)

# カスタム選択用ウィジェット
custom_domain_selector = widgets.SelectMultiple(
    options=domain_options,
    value=['algebra'],
    description="分野選択:",
    style={'description_width': 'initial'},
    rows=6
)

def update_domain_selection(change):
    clear_output(wait=True)
    print(f"🎯 モード: {mode_selector.value}")
    
    if mode_selector.value == 'single':
        display(single_domain_selector, problem_count_input)
        print("\n📝 単一分野の詳細設定")
    elif mode_selector.value == 'all':
        print("\n🔥 全6分野（44,000問）を生成します:")
        for domain, info in domains.items():
            print(f"  • {info['name']}: {info['recommended']:,}問")
    elif mode_selector.value == 'custom':
        display(custom_domain_selector)
        print("\n📋 生成したい分野を複数選択してください")

mode_selector.observe(update_domain_selection, names='value')
update_domain_selection(None)  # 初期表示

## 🚀 データセット生成実行

### ステップ6: 生成パラメータ確認と実行

In [ ]:
import subprocess
import datetime

def execute_generation():
    # 選択されたモデル情報
    selected_model = models[model_selector.value]
    model_path = selected_model['path']
    
    print("🔥 最高難易度数学データセット生成を開始します！")
    print("="*60)
    print(f"📊 選択モデル: {selected_model['name']}")
    print(f"🎯 モデルパス: {model_path}")
    print(f"⚡ 難易度: 研究レベル（IMO・Putnam・PhD級）")
    print(f"🧠 推論深度: 10-20ステップ（4096トークン）")
    print(f"📅 生成開始: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*60)
    
    # 生成モードに応じた実行
    try:
        if mode_selector.value == 'single':
            domain = single_domain_selector.value
            count = problem_count_input.value
            domain_name = domains[domain]['name']
            
            print(f"🎯 単一分野生成: {domain_name}")
            print(f"📝 問題数: {count:,}問")
            print(f"📊 特徴: {domains[domain]['description']}")
            
            # ドメイン別生成実行
            cmd = f"bash scripts/generate_domain_dataset.sh '{model_path}' '{domain}' {count}"
            
        elif mode_selector.value == 'all':
            print("🔥 全分野一括生成（44,000問）")
            total_problems = sum(info['recommended'] for info in domains.values())
            print(f"📊 総問題数: {total_problems:,}問")
            
            for domain, info in domains.items():
                print(f"  • {info['name']}: {info['recommended']:,}問 - {info['description']}")
            
            # 全分野生成実行
            cmd = f"bash scripts/generate_all_math_domains.sh '{model_path}'"
            
        elif mode_selector.value == 'custom':
            selected_domains = custom_domain_selector.value
            print(f"📋 カスタム生成: {len(selected_domains)}分野")
            
            total_count = 0
            for domain in selected_domains:
                domain_info = domains[domain]
                total_count += domain_info['recommended']
                print(f"  • {domain_info['name']}: {domain_info['recommended']:,}問")
            
            print(f"📊 総問題数: {total_count:,}問")
            
            # カスタム生成（各分野を順次実行）
            for domain in selected_domains:
                count = domains[domain]['recommended']
                print(f"\n🎯 生成中: {domains[domain]['name']} ({count:,}問)")
                cmd = f"bash scripts/generate_domain_dataset.sh '{model_path}' '{domain}' {count}"
                
                result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
                if result.returncode != 0:
                    print(f"❌ エラーが発生しました: {result.stderr}")
                    return
                print(f"✅ {domains[domain]['name']} 完了")
            
            print("\n🎉 カスタム生成完了！")
            return
        
        # コマンド実行
        print(f"\n🚀 実行コマンド: {cmd}")
        print("⏳ 生成中... (数時間かかる場合があります)")
        
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("\n🎉 データセット生成完了！")
            print("📁 生成されたファイルはdataディレクトリに保存されました")
            print(result.stdout)
        else:
            print(f"\n❌ エラーが発生しました:")
            print(result.stderr)
            
    except Exception as e:
        print(f"❌ 予期しないエラー: {str(e)}")

# 実行ボタン
execute_button = widgets.Button(
    description="🚀 生成開始",
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px')
)

def on_execute_click(b):
    execute_generation()

execute_button.on_click(on_execute_click)
display(execute_button)

print("\n⚡ 準備完了！上のボタンをクリックして生成を開始してください")

## 📊 生成結果の確認

### ステップ7: データセット確認

In [ ]:
import os
import json
from pathlib import Path

def check_generated_data():
    """生成されたデータセットを確認"""
    data_dir = Path("data")
    
    if not data_dir.exists():
        print("📁 dataディレクトリが見つかりません")
        return
    
    print("📊 生成されたデータセット一覧:")
    print("="*50)
    
    datasets = list(data_dir.iterdir())
    if not datasets:
        print("📝 まだデータセットが生成されていません")
        return
    
    for dataset_path in sorted(datasets):
        if dataset_path.is_dir():
            print(f"\n📂 {dataset_path.name}")
            
            # dataset_info.jsonがあれば情報を表示
            info_file = dataset_path / "dataset_info.json"
            if info_file.exists():
                try:
                    with open(info_file, 'r', encoding='utf-8') as f:
                        info = json.load(f)
                    
                    print(f"  📝 データセット名: {info.get('dataset_name', 'N/A')}")
                    print(f"  🎯 分野: {info.get('domain', 'N/A')}")
                    print(f"  📊 問題数: {info.get('problem_count', 'N/A'):,}")
                    print(f"  🤖 モデル: {info.get('model', 'N/A')}")
                    print(f"  📅 生成日: {info.get('generation_date', 'N/A')}")
                    
                    if 'parameters' in info:
                        params = info['parameters']
                        print(f"  ⚡ 高度推論モード: {params.get('advanced_reasoning_mode', False)}")
                        print(f"  🧠 最大トークン数: {params.get('max_tokens_response', 'N/A')}")
                    
                except Exception as e:
                    print(f"  ❌ 情報読み込みエラー: {e}")
            
            # ファイル一覧
            files = list(dataset_path.glob("*.json"))
            print(f"  📁 ファイル数: {len(files)}")
            for file in sorted(files):
                size_mb = file.stat().st_size / (1024*1024)
                print(f"    • {file.name} ({size_mb:.1f} MB)")

# データ確認ボタン
check_button = widgets.Button(
    description="📊 データ確認",
    button_style='info',
    layout=widgets.Layout(width='150px')
)

def on_check_click(b):
    check_generated_data()

check_button.on_click(on_check_click)
display(check_button)

### ステップ8: サンプルデータ表示

In [ ]:
def show_sample_data(limit=3):
    """生成されたデータのサンプルを表示"""
    data_dir = Path("data")
    
    # 最新のデータセットを探す
    datasets = sorted([d for d in data_dir.iterdir() if d.is_dir()], 
                     key=lambda x: x.stat().st_mtime, reverse=True)
    
    if not datasets:
        print("📝 表示するデータセットがありません")
        return
    
    latest_dataset = datasets[0]
    print(f"🎯 最新データセット: {latest_dataset.name}")
    print("="*50)
    
    # SFTデータ（問題+解答）を探す
    sft_files = list(latest_dataset.glob("*_ins_res.json"))
    
    if not sft_files:
        print("📄 SFTデータファイルが見つかりません")
        return
    
    sft_file = sft_files[0]
    print(f"📄 ファイル: {sft_file.name}")
    
    try:
        with open(sft_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print(f"📊 総データ数: {len(data):,}件")
        print(f"\n🎯 サンプルデータ（最初の{min(limit, len(data))}件）:")
        
        for i, item in enumerate(data[:limit]):
            print(f"\n{'='*20} サンプル {i+1} {'='*20}")
            
            if 'instruction' in item:
                instruction = item['instruction']
                print(f"📝 問題:")
                print(f"{instruction[:300]}{'...' if len(instruction) > 300 else ''}")
            
            if 'output' in item:
                output = item['output']
                print(f"\n🧠 解答:")
                print(f"{output[:500]}{'...' if len(output) > 500 else ''}")
            
            print(f"\n📊 統計:")
            if 'instruction' in item:
                print(f"  • 問題文字数: {len(item['instruction']):,}")
            if 'output' in item:
                print(f"  • 解答文字数: {len(item['output']):,}")
                
    except Exception as e:
        print(f"❌ データ読み込みエラー: {e}")

# サンプル表示ボタン
sample_button = widgets.Button(
    description="📄 サンプル表示",
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

def on_sample_click(b):
    show_sample_data()

sample_button.on_click(on_sample_click)
display(sample_button)

## 📦 データセットダウンロード

### ステップ9: 生成データの圧縮・ダウンロード

In [ ]:
import zipfile
from google.colab import files

def create_download_package():
    """生成されたデータセットを圧縮してダウンロード用に準備"""
    data_dir = Path("data")
    
    if not data_dir.exists() or not list(data_dir.iterdir()):
        print("📁 ダウンロードするデータが見つかりません")
        return
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    zip_filename = f"magpie_math_dataset_{timestamp}.zip"
    
    print(f"📦 データセットを圧縮中: {zip_filename}")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # dataディレクトリ内の全ファイルを追加
            for root, dirs, files in os.walk(data_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, ".")  # 相対パスで保存
                    zipf.write(file_path, arcname)
                    print(f"  ✅ {arcname}")
        
        # ファイルサイズ確認
        size_mb = os.path.getsize(zip_filename) / (1024*1024)
        print(f"\n📊 圧縮完了: {zip_filename} ({size_mb:.1f} MB)")
        
        # ダウンロード開始
        print("🚀 ダウンロードを開始します...")
        files.download(zip_filename)
        
        print("✅ ダウンロード完了！")
        
    except Exception as e:
        print(f"❌ 圧縮エラー: {e}")

# ダウンロードボタン
download_button = widgets.Button(
    description="📦 ダウンロード",
    button_style='success',
    layout=widgets.Layout(width='150px')
)

def on_download_click(b):
    create_download_package()

download_button.on_click(on_download_click)
display(download_button)

print("\n💡 ヒント: 生成完了後、このボタンで全データセットをダウンロードできます")

## 🛠️ トラブルシューティング

### よくある問題と解決方法

#### 1. GPU メモリ不足
```bash
# メモリ使用量を確認
!nvidia-smi

# Colabランタイムを再起動
# ランタイム → ランタイムを再起動
```

#### 2. モデルダウンロードエラー
- Hugging Face Tokenが必要な場合があります
- 代替モデルを選択してください

#### 3. 生成時間が長い
- 問題数を減らして試してください
- 単一分野から開始することを推奨

### サポート
- GitHub Issues: [https://github.com/Ohtani-y/magpie/issues](https://github.com/Ohtani-y/magpie/issues)
- 元論文: [Magpie: Alignment Data Synthesis from Scratch](https://arxiv.org/abs/2406.08464)